In [14]:
import os
import pandas as pd

In [60]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

pd.options.mode.chained_assignment = None

In [79]:
def process_result(dir, to_file=True, show_output=False)->pd.DataFrame:
    '''Forward fill result data
    
    Params:
        dir: directory of input excel file
        to_file: write to output file (default True)
        show_output: display output df (default False)
    
    Returns:
        dataframe (when show_output True)
    '''

    df = pd.read_excel(dir)
    df['Time'] = pd.to_datetime(df['Time'])
    df['Manual'] = 'T'
    start_time = df['Time'].iloc[0]
    end_time = df['Time'].iloc[-1]
    seconds = (end_time - start_time).total_seconds()

    result_df = pd.DataFrame(range(int(seconds) + 1), columns=["Time"]) 
    result_df["Time"] = pd.to_timedelta(result_df["Time"], unit="S") + start_time
    result_df = result_df.merge(df, on="Time", how="left")
    result_df["Manual"].fillna("F", inplace=True)
    result_df.ffill(inplace=True)
    result_df["Time"] = result_df["Time"].astype(str)

    if to_file:
        filename = os.path.basename(dir)
        result_df.to_excel(f"processed_results/{filename}", index=False)

    if show_output:
        return result_df

process_result("results/Q2_TLC00024_c.xlsx")

In [88]:
def process_data_folder(dir):
    '''Process result on folder of data
    
    Params:
        dir: directory of xlsx results
    '''

    for fname in os.listdir(dir):
        fname = os.path.join(dir, fname)

        if not os.path.isfile(fname): continue

        process_result(fname)

    
process_data_folder("results")
